In [ ]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler


In [ ]:

from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [113]:
# Load CSVs
curvelet_df = pd.read_csv('/content/drive/My Drive/6.Input_csvs_for_hybrid/curvelet_preprocessed_celebDF_.csv')
shearlet_df = pd.read_csv('/content/drive/My Drive/6.Input_csvs_for_hybrid/celebDF_preprocessed_shearlet_isomap_100d.csv')



In [114]:
print("📊 Curvelet stats:")
print(curvelet_df.describe().loc[['mean', 'std']])

print("\n📊 Shearlet stats:")
print(shearlet_df.describe().loc[['mean', 'std']])

📊 Curvelet stats:
      var_low_freq  skew_high_freq  kurtosis_mid_freq  mean_diagonal_band  \
mean  2.211281e-17    1.161785e-15      -4.068243e-16       -1.860770e-16   
std   1.000003e+00    1.000003e+00       1.000003e+00        1.000003e+00   

      std_curve_band  energy_low_band  max_coeff_band  entropy_curve_band  \
mean   -8.976861e-16     2.553865e-15    1.176214e-16        1.285993e-17   
std     1.000003e+00     1.000003e+00    1.000003e+00        1.000003e+00   

          iqr_band  contrast_high_band  curvelet_f16  curvelet_f17     label  
mean -4.714264e-16        1.182487e-16 -1.952514e-16  2.608842e-16  0.500000  
std   1.000003e+00        1.000003e+00  1.000003e+00  1.000003e+00  0.500001  

📊 Shearlet stats:
          isomap_1      isomap_2      isomap_3      isomap_4      isomap_5  \
mean  1.035068e-17  5.645825e-18 -8.468737e-18 -5.018511e-18  1.019385e-17   
std   1.000003e+00  1.000003e+00  1.000003e+00  1.000003e+00  1.000003e+00   

          isomap_6      iso

In [10]:
!pip install PyWavelets


In [29]:
import pandas as pd
import numpy as np
import pywt
from tqdm.notebook import tqdm

# Load CSVs
# curvelet_df = pd.read_csv(curvelet_csv_path)
# shearlet_df = pd.read_csv(shearlet_csv_path)

# Merge on 'image_path' and 'label'
df = pd.merge(curvelet_df, shearlet_df, on=["image_path", "label"])
print(f"Merged shape: {df.shape}")

# Get feature column names# If curvelet features are columns 2–13 and shearlet are 14–113 (adjust accordingly)
curvelet_cols = df.columns[2:14]    # 12 features
shearlet_cols = df.columns[14:114]  # 100 features


print(f"Curvelet features: {len(curvelet_cols)}, Shearlet features: {len(shearlet_cols)}")


Merged shape: (181228, 114)
Curvelet features: 12, Shearlet features: 100


In [32]:

# Fusion function
def cross_wavelet_fusion(curvelet, shearlet, wavelet='morl', scales=np.arange(1, 32)):
    curvelet = np.array(curvelet)
    shearlet = np.array(shearlet)

    # Pad curvelet to match length of shearlet
    if len(curvelet) < len(shearlet):
        curvelet = np.pad(curvelet, (0, len(shearlet) - len(curvelet)), 'constant')

    coef_c, _ = pywt.cwt(curvelet, scales, wavelet)
    coef_s, _ = pywt.cwt(shearlet, scales, wavelet)

    cross_energy = np.abs(coef_c * np.conj(coef_s))
    mean_energy = np.mean(cross_energy, axis=1)
    max_energy = np.max(cross_energy, axis=1)

    return np.concatenate([mean_energy, max_energy])


In [15]:
df.dropna(subset=curvelet_cols + shearlet_cols, inplace=True)
print("After dropping NaNs, total rows:", len(df))


After dropping NaNs, total rows: 181228


In [33]:

# Process all rows
fused_features = []
for _, row in tqdm(df.iterrows(), total=len(df)):
    curvelet_feat = row[curvelet_cols].values.astype(np.float32)
    shearlet_feat = row[shearlet_cols].values.astype(np.float32)
    fused_vec = cross_wavelet_fusion(curvelet_feat, shearlet_feat)
    fused_features.append(fused_vec)

# Create final fused DataFrame
fused_df = pd.DataFrame(fused_features, columns=[f"fused_feat_{i}" for i in range(len(fused_features[0]))])
fused_df["image_path"] = df["image_path"]
fused_df["label"] = df["label"]

# Save to Drive
fused_df.to_csv(output_csv_path, index=False)
print(f"✅ Fusion complete. Saved to: {output_csv_path}")

  0%|          | 0/181228 [00:00<?, ?it/s]

OSError: Cannot save file into a non-existent directory: '/content/drive/MyDrive/deepfake_project'

In [34]:
output_csv_path = "/content/drive/MyDrive/hybrid_fused_features.csv"

In [35]:

# Save to Drive
fused_df.to_csv(output_csv_path, index=False)
print(f"✅ Fusion complete. Saved to: {output_csv_path}")

✅ Fusion complete. Saved to: /content/drive/MyDrive/hybrid_fused_features.csv
